# AdAS 2.0
Quick example on how to create an environment for a pipeline of functions.

First, we define the functions and inputs, and then we profile the pipeline on the current device. This creates a profile, saving information about the device, inputs, functions and stats about each run (for now latency and utility).

In [1]:
import glob
import os, sys
import cv2
import types
import time

# Imports for AdAS modules.
sys.path.append('./simulator/')
sys.path.append('./utils/')
from profiler import profile
from entities import AdASIO, AdASFunction

#================================== FUNCTIONS ==================================#
# wrapper for detect_faces in utils
def add_numbers(n1, n2):
    return n1+n2

def square_number(n):
    return n*n

f_add_numbers = AdASFunction(function = add_numbers, 
                            params = {'n2': {10:3, 100:10}})
f_square_number = AdASFunction(function = square_number, 
                            params = {})


#====================================== IO =====================================#
pipeline_inputs = []
for number in range(10):
    pipeline_inputs.append(AdASIO(io_value=number))

#=================================== PROFILE ==================================#
# The first function takes the input from pipeline_inputs, the next ones, take
# as input the output of the previous function, i.e.: square_number(add_numbers(p_input)).
# The output of the last function will be save to results file.
pipeline = [f_add_numbers, f_square_number]
# profile(device id, list of functions to run in order, list of inputs to use, n_repetitions). 
profile('mac_nec', pipeline, pipeline_inputs, 5)
print('Done')

0


100%|██████████| 2/2 [00:10<00:00,  5.03s/it]


1



100%|██████████| 2/2 [00:10<00:00,  5.04s/it]


2


...


8



100%|██████████| 2/2 [00:10<00:00,  5.04s/it]


9



100%|██████████| 2/2 [00:10<00:00,  5.03s/it]


Done


## Check results generated by the profiler
The outputs of the profiler are a .pkl file, which contains the profile with all of its information to be used later by a simulator and a results.csv file, which show all the information obtained by the profiler while running the pipeline.

In [2]:
import pandas as pd

pd.read_csv('results.csv')

,Unnamed: 0,io_value,io_id,io_size,io_format,add_numbers_n2,cpu_idle,memory_available,add_numbers_start,add_numbers_end,square_number_start,square_number_end,output
0,0,0,0,24,<class 'int'>,10,81.2,2183655424,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,100
1,1,0,0,24,<class 'int'>,10,86.0,2176585728,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,100
2,2,0,0,24,<class 'int'>,10,86.4,2178281472,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,100
3,3,0,0,24,<class 'int'>,10,89.2,2180857856,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,100
4,4,0,0,24,<class 'int'>,10,87.8,2167971840,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,9,9,28,<class 'int'>,100,86.3,2189783040,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,11881
96,96,9,9,28,<class 'int'>,100,98.0,2178187264,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,11881
97,97,9,9,28,<class 'int'>,100,91.1,2177396736,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,11881
98,98,9,9,28,<class 'int'>,100,98.3,2177564672,1.584099e+09,1.584099e+09,1.584099e+09,1.584099e+09,11881


## Create a simulator using the profile
We read the .pkl file and create a simulator with it. The simulator can simulate the execution of the pipeline for a given input and given parameters. Note that the combination of parameters and input that we want to simulate, should have been given to the profiler.

In [3]:
from simulator import AdASSimulator

simulator = AdASSimulator('profile_mac_nec.pkl')
simulator.sim(pipeline_inputs[0], {'n2': 100})

defaultdict(list,
            {'utility': 0,
             'latency': [6.198883056640625e-06, 3.0994415283203125e-06]})

## Create an RL environment
Finally, we create an environment using the simulator, giving a latency constrain, a set of inputs and the settings we want for our environment. We can then use it just as any other OpenAI Gym environment.

In [4]:
from environment import AdASEnvironment

'''
Keyword arguments:
    - simulator: an AdASSimulator class, with a profile loaded.
    - latency_target: maximum end-to-end latency allowed for application.
    - inputs: an array of AdASIO that will be used one per step.
    - sample_inputs: True if inputs should be sampled, False if should be used 
                     as list.
    - n_steps: number of steps before environment finishes, if None environment
               never finishes.
    - state_mask: defines array to be returned as state. If None, all are 
                  returned. Options: 'available_cpu', 'latency', 
                  'params', 'utility', 'input'.
                  Note: params will add one element per parameter to the 
                  state array.
'''
env_exc = AdASEnvironment(simulator,
                          latency_target=1.0,
                          inputs=[AdASIO(io_value=_) for _ in range(2)], 
                          sample_inputs=True, 
                          n_steps=100, 
                          state_mask=['params', 'available_cpu', 'utility'])

In [5]:
state, reward, finish, info = env_exc.step(0)